# Data Preparation & Exploration
---
https://www.kaggle.com/code/bertcarremans/data-preparation-exploration/notebook   
캐글 필사 스터디를 진행하면서 기존에 작성된 코드 및 설명을 필사한다.   
## Introduction
이 노트북은 PoreSeguro competition의 데이터로부터 **good insight를 얻는게 목표**이다. 게다가, 모델링을 위해 데이터를 준비하기 위한 몇 가지 tips과 tricks을 제공한다. 이 노트북은 아래 보이는 main sections들로 구성된다.   

1. Visual inspection of your data
2. Defining the metadata
3. Descriptive statistics
4. Handling imbalanced classes
5. Data quality checks
6. Feature engineering
7. Feature selection
8. Feature scaling

## Loading packages

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

## Loading data

In [40]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

## Data at first sight
여기는 competition에 대한 데이터 설명의 일부이다.
- 비슷한 그룹에 속하는 feature들은 feature name에 태그가 지정된다.(ex ind, reg, car, calc)
- feature name들은 접미사에 binary feature인 경우 **bin**을 categorical feature들은 **cat**를 포함한다.
- 이러한 명칭이 없는 feature들은 continuous or ordinal 이다.
- **-1** 값은 관측으로부터 놓친 특징을 나타낸다.
- **target** column은 보험 계약자에 대한 청구가 접수 되었는지 여부이다.

이것은 시작하는데 중요한 정보이다. 모든 것을 확인하기위해 **첫 행과 마지막 행**을 간단히 본다.

In [41]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0


In [42]:
train.tail()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
595207,1488013,0,3,1,10,0,0,0,0,0,1,0,0,0,0,0,13,1,0,0,0.5,0.3,0.692820,10,1,-1,0,1,1,1,1,0,1,31,3,0.374166,0.684631,0.385487,2.645751,0.4,0.5,0.3,3,0,9,0,9,1,12,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,1,0,0,0,0,0,6,1,0,0,0.9,0.7,1.382027,9,1,-1,0,-1,15,0,0,2,1,63,2,0.387298,0.972145,-1.000000,3.605551,0.2,0.2,0.0,2,4,8,6,8,2,12,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,0,0,0,0,0,0,12,1,0,0,0.9,0.2,0.659071,7,1,-1,0,-1,1,1,1,2,1,31,3,0.397492,0.596373,0.398748,1.732051,0.4,0.0,0.3,3,2,7,4,8,0,10,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.9,0.4,0.698212,11,1,-1,0,-1,11,1,1,2,1,101,3,0.374166,0.764434,0.384968,3.162278,0.0,0.7,0.0,4,0,9,4,9,2,11,4,1,4,2,0,1,1,1,0,0
595211,1488027,0,0,1,8,0,0,1,0,0,0,0,0,0,0,0,7,1,0,0,0.1,0.2,-1.000000,7,0,-1,0,-1,0,1,0,2,1,34,2,0.400000,0.932649,0.378021,3.741657,0.4,0.0,0.5,2,3,10,4,10,2,5,4,4,3,8,0,1,0,0,0,0


우리는 실제로 다음을 확인한다.
- binary variables
- category 값이 정수인 categorical variables
- 정수 또는 float 값이 있는 다른 variables 
- -1로 표현된 결측치 variables
- target variable and an ID variable

train data에서 row와 column들의 수를 보자

In [43]:
train.shape

(595212, 59)

우리는 59 variables와 595,212 rows를 가지고 있다. 테스트에 같은 variables 수가 있는지 확인한다.  
또한, train data에 중복 rows들이 있는지 확인한다.

In [44]:
train.drop_duplicates()
train.shape

(595212, 59)

중복 rows는 없다.

In [45]:
test.shape

(892816, 58)

test data에 하나의 variable이 빠져있지만, 이는 target variable 이다.  
우리가 가지고 있는 각각의 타입의 variables이 얼마나 있는지 조사한다.

그리고 후에 우리는 14 categorical variables을 위한 dummy variables들을 만들 수 있다.  
***bin*** variables은 이미 binary 이므로 dummification이 필요없다.  
※ dummification은 categorical variables들을 one hot 인코딩으로 변형하여 variables를 만들어 주는 것이다.

In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              595212 non-null  int64  
 1   target          595212 non-null  int64  
 2   ps_ind_01       595212 non-null  int64  
 3   ps_ind_02_cat   595212 non-null  int64  
 4   ps_ind_03       595212 non-null  int64  
 5   ps_ind_04_cat   595212 non-null  int64  
 6   ps_ind_05_cat   595212 non-null  int64  
 7   ps_ind_06_bin   595212 non-null  int64  
 8   ps_ind_07_bin   595212 non-null  int64  
 9   ps_ind_08_bin   595212 non-null  int64  
 10  ps_ind_09_bin   595212 non-null  int64  
 11  ps_ind_10_bin   595212 non-null  int64  
 12  ps_ind_11_bin   595212 non-null  int64  
 13  ps_ind_12_bin   595212 non-null  int64  
 14  ps_ind_13_bin   595212 non-null  int64  
 15  ps_ind_14       595212 non-null  int64  
 16  ps_ind_15       595212 non-null  int64  
 17  ps_ind_16_

`info()`를 사용하여 우리는 데이터 유형이 integer or float인 것을 볼 수 있다. 이 데이터에는 null 값이 없는 것을 보여준다. 결측치는 -1로 대체되었기 때문에 이는 당연하다.

## Metadata

데이터 관리를 용이하게 하기 위해서, 우리는 DataFrame에 variables의 대한 meta-information을 저장할 것이다. 이는 분석, 시각화, 모델링을 위한 특정 variables을 선택하고 싶을때, 도움이 될 것이다.  

구체적으로 우리는 다음과 같이 저장할 것이다.
- **role**: input, ID, target
- **level**: nominal, interval, ordinal, binary
- **keep**: Ture or False
- **dtype**: int, float, str

In [47]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
    
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == np.float64:
        level = 'interval'
    elif train[f].dtype == np.int64:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # Defining the data type 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

--------------------------------------------------------------------datatype--------------------------------------------------------------------

In [48]:
print(np.int64(1) == 1)
print(np.float64(1) == 1.)
print(np.int64(1).dtype == int)
print(np.int64(1).dtype == np.int_) # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 노션에 정리 필요

True
True
False
False


--------------------------------------------------------------------datatype--------------------------------------------------------------------

In [49]:
meta

,role,level,keep,dtype
varname,,,,
id,id,nominal,False,int64
target,target,binary,True,int64
ps_ind_01,input,ordinal,True,int64
ps_ind_02_cat,input,nominal,True,int64
ps_ind_03,input,ordinal,True,int64
ps_ind_04_cat,input,nominal,True,int64
ps_ind_05_cat,input,nominal,True,int64
ps_ind_06_bin,input,binary,True,int64
ps_ind_07_bin,input,binary,True,int64


예제로 drop 하지 않고 모든 nominal variables들을 추출한다.

In [50]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Index(['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat',
       'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat',
       'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat',
       'ps_car_10_cat', 'ps_car_11_cat'],
      dtype='object', name='varname')

role와 level 당 variables의 수를 아래에 표시한다.

--------------------------------------------------------------------group by(패)--------------------------------------------------------------------  
https://rfriend.tistory.com/383  
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

In [51]:
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index() # @@@@@@@@@@ groupby 정리 (여기부터)

,role,level,count
0,id,nominal,1
1,input,binary,17
2,input,interval,10
3,input,nominal,14
4,input,ordinal,16
5,target,binary,1


In [52]:
pd.DataFrame({'count' : meta.groupby(['level', 'role']).size()}).reset_index()

,level,role,count
0,binary,input,17
1,binary,target,1
2,interval,input,10
3,nominal,id,1
4,nominal,input,14
5,ordinal,input,16


In [53]:
pd.DataFrame({'count' : meta.groupby(['level', 'role'])['role'].size()}).reset_index()

,level,role,count
0,binary,input,17
1,binary,target,1
2,interval,input,10
3,nominal,id,1
4,nominal,input,14
5,ordinal,input,16


In [54]:
meta.groupby(['role', 'level'])['role'].size() ### groupby 정리 내용으로 사용하기 (여기 까지)

role    level   
id      nominal      1
input   binary      17
        interval    10
        nominal     14
        ordinal     16
target  binary       1
Name: role, dtype: int64

--------------------------------------------------------------------group by--------------------------------------------------------------------

## Descriptive statistics
우리는 Dataframe에서 ***describe*** 메소드를 적용할 수 있다. 그러나 id variable과 categorical variables에 대해 mean, std, ...등을 계산하는 것은 의미가 없다. 우리는 이후에 categorical variables을 시각적으로 탐구할 것이다.  
meta file 덕분에 우리는 분석을 원하는 variables을 쉽게 선택할 수 있다. 명확하게 하기위해서, 우리는 data type 당 이 작업을 수행할 것이다.

### Interval variables

In [55]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.610991,0.439184,0.551102,0.379945,0.813265,0.276256,3.065899,0.449756,0.449589,0.449849
std,0.287643,0.404264,0.793506,0.058327,0.224588,0.357154,0.731366,0.287198,0.286893,0.287153
min,0.000000,0.000000,-1.000000,-1.000000,0.250619,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.200000,0.525000,0.316228,0.670867,0.333167,2.828427,0.200000,0.200000,0.200000
50%,0.700000,0.300000,0.720677,0.374166,0.765811,0.368782,3.316625,0.500000,0.400000,0.500000
75%,0.900000,0.600000,1.000000,0.400000,0.906190,0.396485,3.605551,0.700000,0.700000,0.700000
max,0.900000,1.800000,4.037945,1.264911,3.720626,0.636396,3.741657,0.900000,0.900000,0.900000


**reg variables**
- 오직 "ps_reg_03"이 missing values를 가지고 있다. (min 값 확인)
- (min ~ max) 범위가 variables 마다 다르다. 우리는 scaling(StandardScaler)을 적용할 수 있지만, 사용하려는 분류기에 따라 다르다.

**car variables**
- "ps_car_12"와 "ps_car_15"가 missing values를 가지고 있다.
- 이것 또한, scaling을 적용할 수 있다.

**calc variables**
- no missing values
- 이것은 최대값이 0.9이기 때문에 비율인 것 같다.
- 3개의 *_calc* variables 모두 매우 유사한 분포를 가진다.

**전반적으로**, 우리는 interval variables의 범위는 작은것을 볼 수 있다. 아마도 몇가지 변환(e.g. log)이 이미 데이터 익명화를 위해 적용되었을 수도 있다라고 생각한다.

### Ordinal variables

In [56]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

,ps_ind_01,ps_ind_03,ps_ind_14,ps_ind_15,ps_car_11,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,1.900378,4.423318,0.012451,7.299922,2.346072,2.372081,1.885886,7.689445,3.005823,9.225904,2.339034,8.433590,5.441382,1.441918,2.872288,7.539026
std,1.983789,2.699902,0.127545,3.546042,0.832548,1.117219,1.134927,1.334312,1.414564,1.459672,1.246949,2.904597,2.332871,1.202963,1.694887,2.746652
min,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,5.000000,2.000000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000
50%,1.000000,4.000000,0.000000,7.000000,3.000000,2.000000,2.000000,8.000000,3.000000,9.000000,2.000000,8.000000,5.000000,1.000000,3.000000,7.000000
75%,3.000000,6.000000,0.000000,10.000000,3.000000,3.000000,3.000000,9.000000,4.000000,10.000000,3.000000,10.000000,7.000000,2.000000,4.000000,9.000000
max,7.000000,11.000000,4.000000,13.000000,3.000000,5.000000,6.000000,10.000000,9.000000,12.000000,7.000000,25.000000,19.000000,10.000000,13.000000,23.000000


- "ps_car_11"만 missing value를 가지고 있다.
- 이것 또한, scaling을 적용할 수 있다.

### Binary variables

In [57]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

,target,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.036448,0.393742,0.257033,0.163921,0.185304,0.000373,0.001692,0.009439,0.000948,0.660823,0.121081,0.153446,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,0.187401,0.488579,0.436998,0.370205,0.388544,0.019309,0.041097,0.096693,0.030768,0.473430,0.326222,0.360417,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


- 학습 데이터의 target 비율(**priori**)은 3.645%로 심하게 분균형(**strongly imbalanced**)하다
- 평균을 볼때 대부분의 경우에 대부분의 변수의 값이 0이라는 결론을 낼 수 있다.

## Handling imbalanced classes
우리는 target 값이 1보다 0을 가진 것이 더 많다고 말했다. 이것은 좋은 정확도를 가진 모델을 만들 수 있지만 예측에서 가치가 높을 것이다.(대부분의 target이 0이므로 예측을 모두 0으로만해도 실제 target이 1인 것이 적음으로 정확도는 높다) 이 문제를 다루기 위해서 2가지 가능한 전략이 있다.
- oversampling records with target = 1
- undersampling records with target = 0

물론 더 많은 전략들이 있고 MachineLearningMastery.com에서 좋은 overview를 제공한다. 우리는 학습 데이터가 상당히 크기 때문에 **undersampling**을 할 것이다.

<------------------------oversampling, undersampling 정리(패) ------------------------------->  
https://www.kaggle.com/code/rafjaa/resampling-strategies-for-imbalanced-datasets/notebook 참고  

In [58]:
desired_apriori=0.10

# Get the indices per target value
idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

# Get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# Calculate the undersampling rate and resulting number of records with target=0
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersampling records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# Randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# Construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

# Return undersample data frame
train = train.loc[idx_list].reset_index(drop=True)

Rate to undersampling records with target=0: 0.34043569687437886
Number of records with target=0 after undersampling: 195246


<----------------------------- shuffle 정리 ----------------------------->

## Data Quality Checks
### Checking missing values
Missings are represented as -1

In [59]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

Variable ps_ind_02_cat has 103 records (0.05%) with missing values
Variable ps_ind_04_cat has 51 records (0.02%) with missing values
Variable ps_ind_05_cat has 2256 records (1.04%) with missing values
Variable ps_reg_03 has 38580 records (17.78%) with missing values
Variable ps_car_01_cat has 62 records (0.03%) with missing values
Variable ps_car_02_cat has 2 records (0.00%) with missing values
Variable ps_car_03_cat has 148367 records (68.39%) with missing values
Variable ps_car_05_cat has 96026 records (44.26%) with missing values
Variable ps_car_07_cat has 4431 records (2.04%) with missing values
Variable ps_car_09_cat has 230 records (0.11%) with missing values
Variable ps_car_11 has 1 records (0.00%) with missing values
Variable ps_car_14 has 15726 records (7.25%) with missing values
In total, there are 12 variables with missing values


<----------------------------- format 정리 ----------------------------->

- **ps_car_03_cat and ps_car_05_cat**은 많은 missing values를 가지므로 이 variables을 삭제한다.  
- missing values가 있는 categorical variables의 경우, missing value를 -1로 남겨놓을 수 있다.
- **ps_reg_03**(continuous)는 전체 records에 18%의 missing value를 가지고 mean으로 대체한다.
- **ps_car_11**(ordinal)는 오직 5개 records의 missing value를 가지고 mode으로 대체한다.
- **ps_car_12**(continuous)는 오직 1개 records의 missing value를 가지고 mean으로 대체한다.
- **ps_car_14**(continuous)는 전체 records에 7%의 missing value를 가지고 mean으로 대체한다.

In [60]:
# Dropping the variables with too many missing values
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop),'keep'] = False # Updating the meta

# Imputing with the mean or mode
mean_imp = SimpleImputer(missing_values=-1, strategy='mean') #@@@@@@@@@@@ 수정됨
mode_imp = SimpleImputer(missing_values=-1, strategy='most_frequent') # SimpleImputer 정리
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel() # return알아야 ravel() 쓰는거 이해
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

<----------------------------- SimpleImputer 정리 ----------------------------->  
https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html  https://blog.naver.com/PostView.naver?blogId=seokcrew&logNo=221603139044&parentCategoryNo=42&categoryNo=&viewDate=&isShowPopularPosts=true&from=search

### Checking the cardinality of the categorical variables
cardinarlity는 한 variable에서 다른 values의 수를 말한다. 우리는 후에 categorical variables로 부터 dummy variables들을 만들 것이기 때문에, 우리는 variables들이 가진 많은 개별 values들을 확인할 필요가 있다. 우리는 variables 많은 dummy variables을 가지므로 다르게 다뤄야한다.

In [61]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

Variable ps_ind_02_cat has 5 distinct values
Variable ps_ind_04_cat has 3 distinct values
Variable ps_ind_05_cat has 8 distinct values
Variable ps_car_01_cat has 13 distinct values
Variable ps_car_02_cat has 3 distinct values
Variable ps_car_04_cat has 10 distinct values
Variable ps_car_06_cat has 18 distinct values
Variable ps_car_07_cat has 3 distinct values
Variable ps_car_08_cat has 2 distinct values
Variable ps_car_09_cat has 6 distinct values
Variable ps_car_10_cat has 3 distinct values
Variable ps_car_11_cat has 104 distinct values


비록 합리적이지만, 오직 **ps_car_11_cat**이 많은 distinct values를 가진다.

**EDIT** : [nickycan](https://www.kaggle.com/nickycan)가 저자의 첫번째 솔루션이 data Leakage이 일어날 수 있다고 지적하였다. 그는 이것을 다룬 [oliver](https://www.kaggle.com/ogrellier) 커널을 알려주었다. 그러므로 저자는 이부분을 oliver의 커널로 대체했다. Kaggle competitions에 참가함으로써 배울수 있어 정말 좋다.

<------------------------------ Categorical Value Encoding 정리 ------------------------------>  
https://dailyheumsi.tistory.com/120?category=877153

In [66]:
# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    print(trn_series)
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [68]:
train_encoded, test_encoded = target_encode(train["ps_car_11_cat"], 
                             test["ps_car_11_cat"], 
                             target=train.target, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    
# train['ps_car_11_cat_te'] = train_encoded
# train.drop('ps_car_11_cat', axis=1, inplace=True)
# meta.loc['ps_car_11_cat','keep'] = False  # Updating the meta
# test['ps_car_11_cat_te'] = test_encoded
# test.drop('ps_car_11_cat', axis=1, inplace=True)

0          26
1         104
2          15
3          74
4          78
         ... 
216935    104
216936      5
216937    104
216938    104
216939     57
Name: ps_car_11_cat, Length: 216940, dtype: int64


<--- mean encoding --->  
<--- 데이터 정리 --->